# Collect Data From Riot's API

In this code, we will be gathering data from Riot's API using Cassiopeia. Only games from Challenger tier will be collected. 

First, we gather the names of the current Challenger players (NOTE: only current Challenger players can be collected, so the output of this code will change if the list of Challenger players changes on a daily bases). 

Then we collect all the game that those players played for the dates specified for a given patch. So if it was Patch 6.23, then we would collect from 22 Nov 2016 to 7 Dec 2016. 

Each match is saved to its own file and folder along with a .json file using the handy Datreants package. Datreants stores categories and tage in each json file so each match can be searched and found efficiently in the future.

In [1]:
################## Import packages n shit ##################
from cassiopeia import riotapi
from datetime import datetime
import pandas as pd
import datreant.core as dtr

################## Set stuff for Riot API ##################
riotapi.set_region("NA")
riotapi.set_api_key("YOURAPIKEYHERE")

# Here is the list of current Challenger players
masters = [entry.summoner for entry in riotapi.get_challenger()]

################## CHANGE THE PATCH NAME #########################
# Make the folder for the data
matches = dtr.Tree('Data/Matches/Challenger_Patch')
matches.make()

# For each Challenger player:
for master in masters:
    
    ################## CHANGE THE PATCH DATES #########################
    # Get the matches between these two dates
    matchlist = master.match_list(begin_time=datetime(2016, 11, 23), end_time=datetime(2016, 12, 24))
    
    # Now for each match that was collected,
    for game in matchlist:
       
        # Save each match in its own folder with a json file
        match = game.match()
        path0 = matches['Data/Matches/' + str(match.id) + '/']
        path0
        m0 = dtr.Treant(path0)
        m0.path

        # Make the categories for the json file
        m0.categories['MatchID'] = match.id
        m0.categories['VersionPatch'] = match.version
        m0.categories['Year'] = match.creation.year
        m0.categories['Month'] = match.creation.month
        m0.categories['Day'] = match.creation.day
        m0.categories['Hour'] = match.creation.hour
        m0.categories['Duration'] = str(match.duration)
        m0.categories['Map'] = str(match.map)
        m0.categories['Mode'] = str(match.mode)
        m0.categories['Type'] = str(match.type)
        m0.categories['Platform'] = str(match.platform)
        m0.categories['Queue'] = str(match.queue)
        m0.categories['Region'] = str(match.region)
        m0.categories['Season'] = str(match.season)
        m0.categories['Red Ban 1'] = [ban.champion.name for ban in match.red_team.bans][0]
        m0.categories['Red Ban 2'] = [ban.champion.name for ban in match.red_team.bans][1]
        m0.categories['Red Ban 3'] = [ban.champion.name for ban in match.red_team.bans][2]
        m0.categories['Blue Ban 1'] = [ban.champion.name for ban in match.blue_team.bans][0]
        m0.categories['Blue Ban 2'] = [ban.champion.name for ban in match.blue_team.bans][1]
        m0.categories['Blue Ban 3'] = [ban.champion.name for ban in match.blue_team.bans][2]

        # Make empty lists for the data to collect for the match
        p1 = []
        p1champ = []
        p1win = []
        p1side = []
        p1kills = []
        p1deaths = []
        p1assists = []
        p1kda = []
        p1cs = []
        p1spell1 = []
        p1spell2 = []
        p1level = []
        p1fb = []
        p1gold = []
        p1spent = []
        p1item1 = []
        p1item2 = []
        p1item3 = []
        p1item4 = []
        p1item5 = []
        p1item6 = []
        p1mdamage = []
        p1pdamage = []
        p1lane = []
        p1role = []
        
        # For the match, do the following for each participant in the match:
        for participant in match.participants:

            # Fill in the lists made above for each playa
            p1.append(participant.summoner_name)
            p1champ.append(participant.champion.name)
            p1win.append(participant.stats.win)
            p1side.append(participant.side)
            p1kills.append(participant.stats.kills)
            p1deaths.append(participant.stats.deaths)
            p1assists.append(participant.stats.assists)
            p1kda.append(participant.stats.kda)
            p1cs.append(participant.stats.cs)
            p1spell1.append(participant.summoner_spell_d)
            p1spell2.append(participant.summoner_spell_f)
            p1level.append(participant.stats.champion_level)
            p1fb.append(participant.stats.first_blood)
            p1gold.append(participant.stats.gold_earned)
            p1spent.append(participant.stats.gold_spent)
            items = [item.name if item is not None else None for item in participant.stats.items]
            p1item1.append(items[0])
            p1item2.append(items[1])
            p1item3.append(items[2])
            p1item4.append(items[3])
            p1item5.append(items[4])
            p1item6.append(items[5])
            p1mdamage.append(participant.stats.magic_damage_dealt)
            p1pdamage.append(participant.stats.physical_damage_dealt)
            p1lane.append(participant.timeline.lane)
            p1role.append(participant.timeline.role)

        # Save the list column names, put the lists in the columns, transpose so the lists are actually now columns
        columns = ['Player Name', 'Champion', 'Win?', 'Side', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS', 'SSpell 1', 'SSpell 2', 'End Level', 'First Blood?', 'Gold Earned', 'Gold Spent', 'Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Magic Dmg', 'Physical Dmg', 'Lane', 'Role']
        df = pd.DataFrame([p1, p1champ, p1win, p1side, p1kills, p1deaths, p1assists, p1kda, p1cs, p1spell1, p1spell2, p1level, p1fb, p1gold, p1spent, p1item1, p1item2, p1item3, p1item4, p1item5, p1item6, p1mdamage, p1pdamage, p1lane, p1role], index = columns)
        df = df.T

        # Save the file
        m0.draw()
        m0['match.csv']
        filename = "match.csv"
        df.to_csv(m0[filename].abspath)

2380737571/
 +-- Treant.41821fd7-7d89-42be-9fe4-6245cddb1e4d.json
2379429996/
 +-- Treant.1bc153fb-3726-4a2a-9a72-553ca04ec637.json
2378735205/
 +-- Treant.736bbb54-fccc-48d1-9f37-0918b66d9a3e.json
2378753102/
 +-- Treant.9699b564-72d7-48d2-b6ba-89c99707abd5.json
2377256958/
 +-- Treant.f7f81eae-a9a2-4918-94d1-ace6dc5aeffb.json
2377164730/
 +-- Treant.173c4fde-1950-40b2-a15e-b3827b8f38d4.json
2377180436/
 +-- Treant.ec9c9b50-a411-42c7-864f-29b22b185cc7.json
2377026656/
 +-- Treant.c2208dc4-ea08-450b-8914-c3aad9f61b39.json
2377034621/
 +-- Treant.4d270e51-f276-499a-ac0b-08d5c9f92f9c.json


KeyboardInterrupt: 

# Error Management

Most of the time, Riot's API will throw errors for a variety of reasons. This code is set up to try to collect data a few more times beofre it quits. But it still quits a lot to prevent my API key from getting blacklisted.

This bit of code outputs the list of Challenger players. This way I can see which player was last outputted, and restart the code from there. Note that because the list of Challenger players is constantly changing, this method isn't totally amazing.

In [49]:
from cassiopeia import riotapi
from datetime import datetime
import pandas as pd
import datreant.core as dtr

riotapi.set_region("NA")
riotapi.set_api_key("94e831f6-ef9f-4823-81fc-cfc9342f4428")

masters = [entry.summoner for entry in riotapi.get_challenger()]

In [50]:
for index, master in enumerate(masters):
    print(index, master)

0 vìper
1 moon o
2 Entranced
3 Shiphtur
4 Révenge
5 good player95
6 Imaqtpie
7 Bobi Boss yoyo
8 C9 Sneaky
9 TheOddOrange
10 5tunt
11 Témpest
12 Keegun
13 Jurassiq
14 KiNG Nidhogg
15 Adrian Riven
16 FZ3R0
17 Celend
18 Doublelift
19 Hanjaro
20 Cris
21 Stiches
22 lanjunglerxd
23 TRUMPSSMALLHANDS
24 TQS Zuardian
25 FlREFLY
26 Liquid fabbbyyy
27 aShady
28 SWKM
29 GodMeru
30 lenny the lion
31 Linsaníty
32 mulgokizary
33 Aphromoo Lite
34 Walnut K
35 Pyrites
36 Abou222
37 Dhokla
38 Nëbu
39 Sheiden
40 RealTeemo
41 Winter is back
42 Flaresz
43 rovex
44 fallenbandit
45 foggedftw2
46 Piggy Kitten
47 Soligo
48 Lose to win 
49 Grande Donger
50 ChanceIor
51 LL Stylish
52 KarasMai
53 Contractz
54 sEIYA HA
55 Grigne
56 açê
57 Tower of God
58 BukZacH
59 VXlN wedabest
60 TDK Tr4nce
61 Gankr
62 Nwok
63 DZG Doomtrobo
64 my bad team
65 Anthøny
66 ZTG SolidSnake
67 rhubarbs
68 StumpeyAKAMisty
69 Alex ïch
70 C9 Meteos
71 Pauł
72 OoR4s3Ng4N4rUtOo
73 andtheknee
74 Saskioo
75 Pikabi
76 Erry
77 WatchThePower
78 B